# Qaintellect Training Example

In this example, we will demonstrate the optimisation of a quantum circuit using Qaintellect's Flux integration.

In [1]:
include("../src/Qaintellect.jl")
using .Qaintellect

using Flux, LinearAlgebra, Test
using Flux.Optimise: update!
using IterTools: ncycle

┌ Warning: `haskey(::TargetIterator, name::String)` is deprecated, use `Target(; name = name) !== nothing` instead.
│   caller = llvm_compat(::VersionNumber) at compatibility.jl:176
└ @ CUDAnative /home/ga53vuw/.julia/packages/CUDAnative/C91oY/src/compatibility.jl:176


## Task: Creating a Flux layer using Qaintessent
Here, we create a complex circuit with $N$ qubits and different input parameters. We then train these parameters using the Flux framework.

In [2]:
# construct parametrized circuit
N = 4
rz = RzGate(1.5π)
ps = PhaseShiftGate(0.3)
ry = RyGate(√2)
n = randn(Float64, 3)
n = n/norm(n)
rg = RotationGate(0.2π, n)

cgc = CircuitGateChain{N}([
    single_qubit_circuit_gate(3, HadamardGate(), N),
    controlled_circuit_gate(2, (1,4), rz, N),
    two_qubit_circuit_gate(2, 3, SwapGate(), N),
    single_qubit_circuit_gate(3, ps, N),
    single_qubit_circuit_gate(3, rg, N),
    single_qubit_circuit_gate(1, ry, N),
])
meas = MeasurementOps{N}([Matrix{Float64}(I, 2^N, 2^N), Hermitian(randn(ComplexF64, 2^N, 2^N))])

c = Circuit(cgc, meas)


    4 ————————•—————————————————————
              |                     
    3 —[H ]—————————x————[Pϕ]——[Rθ]—
              |     |               
    2 ———————[Rz]———x———————————————
              |                     
    1 ————————•————————————————[Ry]—


In [3]:
function output(ψ, Δ)
    @assert length(ψ) == length(Δ)
    dot(ψ, Δ)
end

output (generic function with 1 method)

In [4]:
# input quantum state
ψ = randn(ComplexF64, 2^N)
ψ = ψ/norm(ψ)

# fictitious gradients of cost function with respect to circuit output
Δ = [0.3, -1.2]
e = 0.65

0.65

In [5]:
# set up model
model(ψ) = output(c(ψ), Δ)

# create loss function
loss(x,y) = Flux.mse(model(x), y)

# gather parameters from Circuit
paras = Flux.params(c)

# freeze parameter ry.θ, equivalent to delete!(paras, Qaintessent.get_trainable(ry))
delete!(paras, ry.θ)

# set up data for training
data = ncycle([(ψ, e)], 150)

# define optimizer
opt = Descent(1)

# define evaluation function
evalcb() = @show(loss(ψ, e))

evalcb (generic function with 1 method)

In [6]:
println("Initial model evaluation: " * string(model(ψ)) * ", Target: " * string(e))

Flux.train!(loss, paras, data, opt, cb=Flux.throttle(evalcb, 0.01))

println("Final model evaluation: " * string(model(ψ)) * ", Target: " * string(e))

Initial model evaluation: 2.2756075150437085, Target: 0.65
loss(ψ, e) = 2.3101834739698632
loss(ψ, e) = 1.732986943647594e-9
loss(ψ, e) = 2.702102565522235e-23
loss(ψ, e) = 0.0
loss(ψ, e) = 0.0
loss(ψ, e) = 0.0
loss(ψ, e) = 0.0
loss(ψ, e) = 0.0
loss(ψ, e) = 0.0
loss(ψ, e) = 0.0
loss(ψ, e) = 0.0
loss(ψ, e) = 0.0
loss(ψ, e) = 0.0
loss(ψ, e) = 0.0
Final model evaluation: 0.65, Target: 0.65
